In [3]:
import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.utils import column_or_1d
from keras.regularizers import l2
from keras.models import Model
from keras.optimizers import SGD
from sklearn import svm
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix

import util

try:
    import cPickle as pickle
except:
    import pickle

seed = 7
np.random.seed(seed)
# path to the model weights file.
weights_path = '../dataset/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 224, 224
nb_classes = 10

nb_train_samples = 1700
nb_validation_samples = 300
nb_epoch = 10


def get_layer_weights(weights_file=None, layer_name=None):
    if not weights_file or not layer_name:
        return None
    else:
        g = weights_file[layer_name]
        weights = [g[p] for p in g]
        print 'Weights for "{}" are loaded'.format(layer_name)
        return weights

def load_data():
    # load your data using this function
    f = open("../dataset/myfood10-224.pkl", 'rb')
    d = pickle.load(f)
    data = d['trainFeatures']
    labels = d['trainLabels']
    lz = d['labels']
    data = data.reshape(data.shape[0], 3, 224, 224)
    #data = data.transpose(0, 2, 3, 1)

    return data,labels,lz

def save_bottlebeck_features(X_train, X_test, y_train, y_test):
    weights_path='../dataset/vgg16_weights.h5'
    # build the VGG16 network
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))


    assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
    f = h5py.File(weights_path)
    for k in range(f.attrs['nb_layers']):
        if k >= len(model.layers):
            # we don't look at the last (fully-connected) layers in the savefile
            break
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        model.layers[k].set_weights(weights)
    f.close()
    print('Model loaded.')

    bottleneck_features_train = model.predict(X_train, batch_size=32)
    np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)


    bottleneck_features_validation = model.predict(X_test, batch_size=32)
    np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.jet):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(10)
    plt.xticks(tick_marks, ['AisKacang' , 'AngKuKueh' , 'ApamBalik' , 'Asamlaksa' , 'Bahulu' , 'Bakkukteh',
    'BananaLeafRice' , 'Bazhang' , 'BeefRendang' , 'BingkaUbi'], rotation=45)
    plt.yticks(tick_marks, ['AisKacang' , 'AngKuKueh' , 'ApamBalik' , 'Asamlaksa' , 'Bahulu' , 'Bakkukteh',
    'BananaLeafRice' , 'Bazhang' , 'BeefRendang' , 'BingkaUbi'])
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
'''
def train_top_model(y_train, y_test):
    train_data = np.load(open('bottleneck_features_train.npy' , 'rb'))
    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))


    print train_data.shape
    print validation_data.shape
    print "Training CNN.."

    input = None
    shape=train_data.shape[1:]
    output = None
    W_regularizer=True
    weights_file_path = None
    weights_file = None

    model = Sequential()
    model.add(Flatten(name='flatten', input_shape=shape))
    W_regularizer = l2(1e-2)
    model.add(Dense(4096, activation='relu',W_regularizer=W_regularizer))
    model.add(Dropout(0.6))
    W_regularizer = l2(1e-2)
    model.add(Dense(4096, activation='relu',W_regularizer=W_regularizer))
    model.add(Dense(nb_classes,activation='softmax'))

    rms = RMSprop(lr=5e-4, rho=0.9, epsilon=1e-08, decay=0.01)
    model.compile(optimizer=rms, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, y_train,
              nb_epoch=nb_epoch, batch_size=32,
              validation_data=(validation_data, y_test))

    Y_pred = model.predict(validation_data)
    print(Y_pred)
    y_pred = np.argmax(Y_pred, axis=1)
    print(y_pred)
'''
if __name__ == "__main__":
    print "Loading data.."
    data, labels, lz = load_data()
    data = data.astype('float32')
    data /= 255
    lz = np.array(lz)
    print "Data loaded !"

    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.15, random_state=seed)
    print X_train.shape
    print X_test.shape
    print "Test train splitted !"

    #save_bottlebeck_features(X_train, X_test, y_train, y_test)
    #train_top_model(y_train, y_test)
    train_data = np.load(open('bottleneck_features_train.npy' , 'rb'))
    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))


    print train_data.shape
    print validation_data.shape
    print "Training CNN.."

    input = None
    shape=train_data.shape[1:]
    output = None
    W_regularizer=True
    weights_file_path = None
    weights_file = None

    model = Sequential()
    model.add(Flatten(name='flatten', input_shape=shape))
    W_regularizer = l2(1e-2)
    model.add(Dense(4096, activation='relu',W_regularizer=W_regularizer))
    model.add(Dropout(0.6))
    W_regularizer = l2(1e-2)
    model.add(Dense(4096, activation='relu',W_regularizer=W_regularizer))
    model.add(Dense(nb_classes,activation='softmax'))

    rms = RMSprop(lr=5e-4, rho=0.9, epsilon=1e-08, decay=0.01)
    model.compile(optimizer=rms, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, y_train,
              nb_epoch=nb_epoch, batch_size=32,
              validation_data=(validation_data, y_test))

    Y_pred = model.predict(validation_data)
    print(Y_pred)
    y_pred = np.argmax(Y_pred, axis=1)
    print(y_pred)


Loading data..
Data loaded !
(1700, 3, 224, 224)
(300, 3, 224, 224)
Test train splitted !
(1700, 512, 7, 7)
(300, 512, 7, 7)
Training CNN..
Train on 1700 samples, validate on 300 samples
Epoch 1/10
1700/1700 [==============================] - 10s - loss: 42.2689 - acc: 0.0953 - val_loss: 14.2377 - val_acc: 0.1167
Epoch 2/10
1700/1700 [==============================] - 10s - loss: 13.3215 - acc: 0.1194 - val_loss: 2.0857 - val_acc: 0.2400
Epoch 3/10
1700/1700 [==============================] - 10s - loss: 1.9633 - acc: 0.4524 - val_loss: 1.2619 - val_acc: 0.5800
Epoch 4/10
1700/1700 [==============================] - 10s - loss: 1.3046 - acc: 0.6659 - val_loss: 1.1551 - val_acc: 0.6467
Epoch 5/10
1700/1700 [==============================] - 10s - loss: 1.0347 - acc: 0.7459 - val_loss: 1.3549 - val_acc: 0.5467
Epoch 6/10
1700/1700 [==============================] - 10s - loss: 0.8618 - acc: 0.8100 - val_loss: 0.5355 - val_acc: 0.8067
Epoch 7/10
1700/1700 [==============================] 

In [12]:
target_names = ['class 0(BIKES)', 'class 1(CARS)', 'class 2(HORSES)','3','3','3','3','3','3','3']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

                 precision    recall  f1-score   support

 class 0(BIKES)       1.00      0.08      0.15       300
  class 1(CARS)       0.00      0.00      0.00         0
class 2(HORSES)       0.00      0.00      0.00         0
              3       0.00      0.00      0.00         0
              3       0.00      0.00      0.00         0
              3       0.00      0.00      0.00         0
              3       0.00      0.00      0.00         0
              3       0.00      0.00      0.00         0
              3       0.00      0.00      0.00         0
              3       0.00      0.00      0.00         0

    avg / total       1.00      0.08      0.15       300

[[25 28 36 23 31 31 32 33 30 31]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0